In [1]:
use ../data/litigclean_eps.dta, clear

In [2]:
*outsheet using "litigclean_eps.csv", comma

In [3]:
drop id
egen id = group(country)

In [4]:
xtset id year


Panel variable: id (strongly balanced)
 Time variable: year, 1990 to 2018
         Delta: 1 unit


In [5]:
xtreg lnghg lngdp lngdp2 lnpop lnlaw litigation eps i.year,fe robust
outreg2 using Table6.xls, replace ctitle (EIT-FE) drop   (i.year o._ _cons)  stats(coef, tstat, blank)



Fixed-effects (within) regression               Number of obs     =        359
Group variable: id                              Number of groups  =         18

R-squared:                                      Obs per group:
     Within  = 0.7581                                         min =         19
     Between = 0.8887                                         avg =       19.9
     Overall = 0.8881                                         max =         20

                                                F(17, 17)         =          .
corr(u_i, Xb) = -0.6630                         Prob > F          =          .

                                    (Std. err. adjusted for 18 clusters in id)
------------------------------------------------------------------------------
             |               Robust
       lnghg | Coefficient  std. err.      t    P>|t|     [95% conf. interval]
-------------+----------------------------------------------------------------
       lngdp |   2.347226  

In [6]:
xtreg lnghg lngdp lngdp2 lnpop lnlaw litigation eps_* i.year,fe robust
outreg2 using Table6.xls, append ctitle (EIT-FE) drop   (i.year o._ _cons)  stats(coef, tstat, blank)



Fixed-effects (within) regression               Number of obs     =        359
Group variable: id                              Number of groups  =         18

R-squared:                                      Obs per group:
     Within  = 0.7619                                         min =         19
     Between = 0.8891                                         avg =       19.9
     Overall = 0.8883                                         max =         20

                                                F(17, 17)         =          .
corr(u_i, Xb) = -0.7231                         Prob > F          =          .

                                    (Std. err. adjusted for 18 clusters in id)
------------------------------------------------------------------------------
             |               Robust
       lnghg | Coefficient  std. err.      t    P>|t|     [95% conf. interval]
-------------+----------------------------------------------------------------
       lngdp |   2.249628  

In [7]:
xtreg lnghg lngdp lngdp2 lnpop litigation c.lnlaw##c.eps_* i.year,fe robust
outreg2 using Table6.xls, append ctitle (EIT-FE) drop   (i.year o._ _cons)  stats(coef, tstat, blank)



Fixed-effects (within) regression               Number of obs     =        359
Group variable: id                              Number of groups  =         18

R-squared:                                      Obs per group:
     Within  = 0.7668                                         min =         19
     Between = 0.8939                                         avg =       19.9
     Overall = 0.8935                                         max =         20

                                                F(17, 17)         =          .
corr(u_i, Xb) = -0.4114                         Prob > F          =          .

                                    (Std. err. adjusted for 18 clusters in id)
------------------------------------------------------------------------------
             |               Robust
       lnghg | Coefficient  std. err.      t    P>|t|     [95% conf. interval]
-------------+----------------------------------------------------------------
       lngdp |   2.590383  

In [8]:
xtreg lnghg lngdp lngdp2 lnpop lnlaw i.litigation##c.eps_* i.year,fe robust
outreg2 using Table6.xls, append ctitle (EIT-FE) drop   (i.year o._ _cons)  stats(coef, tstat, blank)



Fixed-effects (within) regression               Number of obs     =        359
Group variable: id                              Number of groups  =         18

R-squared:                                      Obs per group:
     Within  = 0.7800                                         min =         19
     Between = 0.8897                                         avg =       19.9
     Overall = 0.8891                                         max =         20

                                                F(17, 17)         =          .
corr(u_i, Xb) = -0.6572                         Prob > F          =          .

                                    (Std. err. adjusted for 18 clusters in id)
------------------------------------------------------------------------------
             |               Robust
       lnghg | Coefficient  std. err.      t    P>|t|     [95% conf. interval]
-------------+----------------------------------------------------------------
       lngdp |   2.216569  